# Вычислительная задача определения предварительной орбиты по трем наблюдениям методом Гаусса.

In [1]:
import numpy as np
from sympy.solvers import solve
from sympy import Symbol

In [2]:
## 3 момента наблюдения 21.02 , 12.03 , 1.04
Alpha_ = [5.75964, 5.90033, 6.17065]  #h
Delta_ = [19.48434, 19.70853, 19.87678] #grad
## геоцентрические прямоугольные экваториальные координаты Солнца
X = [0.86926948, 0.98297332, 0.97936910]
Y = [-0.43236176, -0.13314763, 0.18209277]
Z = [-0.18743197,  -0.05771944,  0.07893439]
## промежутки времени между наблюдениями
tau1 = 20 # сут
tau2 = 20 # сут
tau = tau1+tau2
kappa = 0.017202 

In [3]:
## переведём в радианы
Alpha = [Alpha_[i]*np.pi / 12. for i in range(3)]
Delta = [Delta_[i]*np.pi / 180. for i in range(3)]

In [4]:
Lambda = [np.cos(Delta[i])*np.cos(Alpha[i]) for i in range(3)]
Mu = [np.cos(Delta[i])*np.sin(Alpha[i]) for i in range(3)]
Nu = [np.sin(Delta[i]) for i in range(3)]


In [5]:
D = np.array([[Lambda[1], Lambda[0], Lambda[2]],[Mu[1],Mu[0],Mu[2]],[Nu[1],Nu[0],Nu[2]]])
detD = np.linalg.det(D)
U = [X[i]*(Mu[0]*Nu[2]-Mu[2]*Nu[0]) + Y[i]*(Nu[0]*Lambda[2]-Nu[2]*Lambda[0])
                                + Z[i]*(Mu[2]*Lambda[0]-Mu[0]*Lambda[2]) for i in range(3)]
n0_1 = tau2/tau
n0_2 = tau1/tau
c_1 = kappa**2 * tau1 * tau2 * (1+n0_1)/6
c_2 = kappa**2 * tau1 * tau2 * (1+n0_2)/6

In [6]:
P = (U[1]-n0_1*U[0]-n0_2*U[2]) / detD
Q = (c_1*U[0]+c_2*U[2]) / detD
C = -(Lambda[1]*X[1]+Mu[1]*Y[1]+Nu[1]*Z[1])
R_square = X[1]**2+Y[1]**2+Z[1]**2

In [7]:
## Решим систему уравнений Лагранжа и получим гелиоцентрические и геоцентрические расстояния
x = Symbol('x')
roots = []
funct = x**8 - (P*x**3-Q)**2 - 2*C*x**3*(P*x**3-Q) - R_square*x**6
roots = solve(funct, x)

k = [(P*root**3-Q) / root**3 for root in roots]
            
print(roots)                   
print(k)
           

[-3.16437636451661, 2.99379503411122, -0.541467322616619 - 0.732079189340939*I, -0.541467322616619 + 0.732079189340939*I, -0.348598542988237 - 0.891134363386735*I, -0.348598542988237 + 0.891134363386735*I, 0.975356530807553 - 0.0214741539657337*I, 0.975356530807553 + 0.0214741539657337*I]
[2.88612149780267, 2.70604369069965, (-2.61635646683655 + 2.80354955209334*(-0.541467322616619 - 0.732079189340939*I)**3)/(-0.541467322616619 - 0.732079189340939*I)**3, (-2.61635646683655 + 2.80354955209334*(-0.541467322616619 + 0.732079189340939*I)**3)/(-0.541467322616619 + 0.732079189340939*I)**3, (-2.61635646683655 + 2.80354955209334*(-0.348598542988237 - 0.891134363386735*I)**3)/(-0.348598542988237 - 0.891134363386735*I)**3, (-2.61635646683655 + 2.80354955209334*(-0.348598542988237 + 0.891134363386735*I)**3)/(-0.348598542988237 + 0.891134363386735*I)**3, (-2.61635646683655 + 2.80354955209334*(0.975356530807553 - 0.0214741539657337*I)**3)/(0.975356530807553 - 0.0214741539657337*I)**3, (-2.616356466

In [8]:
## Выберем пару положительных значений для расстояний
r = 2.99379503411122
ro = 2.70604369069965

In [9]:
n1 = n0_1 + c_1 / r**3
n2 = n0_2 + c_2 / r**3

In [10]:
## Найдём геоцентрические кооринаты, для двух других моментов, для этого нужно решить систему линейных уравнений
from sympy.solvers.solvers import solve_linear_system_LU
from sympy import Matrix
from sympy.abc import x, y, z
solve_linear_system_LU(Matrix([
       [n1*Lambda[0], -Lambda[1], n2*Lambda[2], n1*X[0]-X[1]+n2*X[2]],
       [n1*Mu[0], -Mu[1], n2*Mu[2], n1*Y[0]-Y[1]+n2*Y[2]],
       [n1*Nu[0], -Nu[1], n2*Nu[2], n1*Z[0]-Z[1]+n2*Z[2]]]), [x, y, z])

{x: 2.44261930042480, y: 2.70604369069964, z: 2.97967362945131}

In [11]:
ro_ = [2.44261930042480, 2.70604369069964, 2.97967362945131]

In [12]:
## Найдём гелиоцентрические координаты для всех моментов
x_ = [ro_[i]*Lambda[i]-X[i] for i in range(3)]
y_ = [ro_[i]*Mu[i]-Y[i] for i in range(3)]
z_ = [ro_[i]*Nu[i]-Z[i] for i in range(3)]
print(x_)
print(y_)
print(z_)

[-0.7244628242415352, -0.9165069155145013, -1.1045170557809623]
[2.7305412642895397, 2.679805003869605, 2.617273740150062]
[1.0021656966366694, 0.9702932140948198, 0.9341500433330024]


In [76]:
r1 = np.array([x_[0], y_[0], z_[0]])
r2 = np.array([x_[1], y_[1], z_[1]])
r3 = np.array([x_[2], y_[2], z_[2]])
print(r1, r3)

[-0.72446282  2.73054126  1.0021657 ] [-1.10451706  2.61727374  0.93415004]


In [77]:
c1 = np.linalg.norm(np.cross(r2,r3)) / np.linalg.norm(np.cross(r1,r3))
c3 = np.linalg.norm(np.cross(r1,r2)) / np.linalg.norm(np.cross(r1,r3))
p = (c1*np.linalg.norm(r1) + c3*np.linalg.norm(r3) - np.linalg.norm(r2))/(c1+c3-1) 
print(p)

3.0739221509814847


In [78]:
## Сначала введём вспомогательный вектор
sigma = np.dot(r1,r3)/np.linalg.norm(r1)**2

r0 = r3 - sigma*r1 
print(r0)

[-0.38828731 -0.0822366  -0.05662666]


In [80]:
f_double = np.arctan(np.linalg.norm(r0)*np.linalg.norm(r1)/np.dot(r1,r3))

q1 = p/np.linalg.norm(r1) - 1
q2 = p/np.linalg.norm(r3) - 1

e_sin_theta1 = q1 / np.tan(f_double) - q2 / np.sin(f_double)
print(f_double*180/np.pi/2)
print(q1,q2)

3.8523415957317435
0.025493480926748635 0.02791722495013671


In [82]:
## Найдём истиные аномалии для 1 и 3 наблюдений, а также эксцентриситет и большую полуось
theta1 = np.arctan2(e_sin_theta1,q1) # rad
theta2 = theta1 + f_double
e = q1 / np.cos(theta1)
a = p/(1-e**2)
print(a,e)
print(360.+theta1*180./np.pi,360+theta2*180./np.pi)

3.0771278083874143 0.03227645178708854
322.17128028649825 329.87596347796176


In [86]:
## Найдём эксцентрические аномалии
E1 = 2 * np.arctan(np.sqrt((1-e)/(1+e)) * np.tan(theta1/2))
E2 = 2 * np.arctan(np.sqrt((1-e)/(1+e)) * np.tan(theta2/2))

## Средняя аномалия
M1 = E1 - e * np.sin(E1)
M0 = M1
print(360.+E1*180./np.pi,360+E2*180./np.pi)
print(360.+M0*180./np.pi)

323.2914416268893 330.7915195088296
324.3968539449184


In [87]:
### Будем искать наклон, аргумент перицентра и долготу восходящего узла 

## Сначала найдём векторные элементы P и Q
P = [r1[i]/np.linalg.norm(r1)*np.cos(theta1) - r0[i]/np.linalg.norm(r0)*np.sin(theta1) for i in range(3)]
Q = [r1[i]/np.linalg.norm(r1)*np.sin(theta1) + r0[i]/np.linalg.norm(r0)*np.cos(theta1) for i in range(3)]
eps = 23.43674167*np.pi/180.

omega = np.arctan2((P[2]*np.cos(eps) - P[1]*np.sin(eps)),(Q[2]*np.cos(eps) - Q[1]*np.sin(eps)))
i = np.arcsin((P[2]*np.cos(eps) - P[1]*np.sin(eps))/ np.sin(omega))
Omega = np.arctan2((P[1]*np.cos(omega)-Q[1]*np.sin(omega))/np.cos(eps),P[0]*np.cos(omega)-Q[0]*np.sin(omega))
print(P)
print(Q)                 

[-0.7848761889207359, 0.5937015190188151, 0.17744823014055466]
[-0.6167327250984217, -0.7206945323058915, -0.31660722811879133]


In [88]:
## Выведем все параметры орбиты:

print('эксцентриситет e =',e)
print('большая полуось a =',a)
print('средняя аномалия M0 =',360+M0*180./np.pi)
print('наклон орбиты i =',i*180./np.pi)
print('аргумент перицентра ω =',360.+omega*180./np.pi)
print('долгота восходящего узла Ω =',360.+Omega*180./np.pi)

эксцентриситет e = 0.03227645178708854
большая полуось a = 3.0771278083874143
средняя аномалия M0 = 324.3968539449184
наклон орбиты i = 4.210931054835446
аргумент перицентра ω = 267.0017909689504
долгота восходящего узла Ω = 234.91186760312098
